<a href="https://colab.research.google.com/github/avrymi-asraf/IDL-huji/blob/main/ex3_IDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:

!pip install torchtext
# !pip install torch
# ! pip install pandas
# ! pip install numpy
!wget -O /content/IMDB_Dataset.csv https://raw.githubusercontent.com/avrymi-asraf/IDL-huji/main/ex3/IMDB%20Dataset.csv


#https://raw.githubusercontent.com/avrymi-asraf/IDL-huji/main/ex3/IMDB%20Dataset.csv

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [6]:
import torch
# from torchtext.legacy.data import Field
import torchtext as tx
from torchtext.vocab import GloVe
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
import re
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch as tr
from torch.nn.functional import pad
import torch.nn as nn
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprec

In [7]:
########################################################################
########################################################################
##
##                     BEST TEAM EVER
                                            ##                     ##
##                                                                    ##
########################################################################
########################################################################


# import loader as ld


batch_size = 32
output_size = 1  # should be 2 for RNN and GRU
#hidden_size = 64        # to experiment with

run_recurrent = False    # else run Token-wise MLP
use_RNN = False         # otherwise GRU
atten_size = 0          # atten > 0 means using restricted self atten

reload_model = True
num_epochs = 10
learning_rate = 0.001
test_interval = 50

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# MODE = "MLP"
#MODE = "RNN"
#MODE = "GRU"
#MODE = "MLP_atten"

In [9]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAX_LENGTH = 100
embedding_size = 100
Train_size=30000



def review_clean(text):
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # remove non alphabetic character
    text = re.sub(r'https?:/\/\S+', ' ', text)  # remove links
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)  # remove singale char
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def tokinize(s):
    s = review_clean(s).lower()
    splited = s.split()
    return splited[:MAX_LENGTH]


def load_data_set(load_my_reviews=False):
    data=pd.read_csv("/content/IMDB_Dataset.csv")
    train_data=data[:Train_size]
    train_iter=ReviewDataset(train_data["review"],train_data["sentiment"])
    test_data=data[Train_size:]
    if load_my_reviews:
        my_data = pd.DataFrame({"review": my_test_texts, "sentiment": my_test_labels})
        # test_data=test_data.append(my_data)
        test_data = pd.concat([test_data, my_data], ignore_index=True)
    test_data=test_data.reset_index(drop=True)
    test_iter=ReviewDataset(test_data["review"],test_data["sentiment"])
    return train_iter, test_iter


embadding = GloVe(name='6B', dim=embedding_size)
tokenizer = get_tokenizer(tokenizer=tokinize)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:23<00:00, 17286.31it/s]


In [10]:



def preprocess_review(s):
    cleaned = tokinize(s)
    embadded = embadding.get_vecs_by_tokens(cleaned)
    if embadded.shape[0] != 100 or embadded.shape[1] != 100:
        embadded = torch.nn.functional.pad(embadded, (0, 0, 0, MAX_LENGTH - embadded.shape[0]))
    return torch.unsqueeze(embadded, 0)


def preprocess_label(label):
    return [0.0, 1.0] if label == "negative" else [1.0, 0.0]


def collact_batch(batch):
    label_list = []
    review_list = []
    embadding_list=[]
    for  review,label in batch:
        label_list.append(preprocess_label(label))### label
        review_list.append(tokinize(review))### the  actuall review
        processed_review = preprocess_review(review).detach()
        embadding_list.append(processed_review) ### the embedding vectors
    label_list = torch.tensor(label_list, dtype=torch.float32).reshape((-1, 2))
    embadding_tensor= torch.cat(embadding_list)
    return label_list.to(device), embadding_tensor.to(device) ,review_list


##########################
# ADD YOUR OWN TEST TEXT #
##########################

my_test_texts = []
my_test_texts.append(" Never did I have such a good time in a movie in my whole life! Had such a good time playing on my smartphone the whole movie.")
my_test_texts.append(" The movie was so captivating that I was completely absorbed in it, causing me to return home late to my wife, who was understandably upset that I lost track of time.")
my_test_texts.append("I was so engrossed in the movie's plot twists that I lost track of time and ended up missing an important appointment, bad luck.")
my_test_labels = ["neg", "pos", "pos"]

##########################
##########################


class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, review_list, labels):
        'Initialization'
        self.labels = labels
        self.reviews = review_list

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        X = self.reviews[index]
        y = self.labels[index]
        return X, y



def get_data_set(batch_size, toy=False):
        train_data, test_data = load_data_set(load_my_reviews=toy)
        train_dataloader = DataLoader(train_data, batch_size=batch_size,
                                      shuffle=True, collate_fn=collact_batch)
        test_dataloader = DataLoader(test_data, batch_size=batch_size,
                                     shuffle=True, collate_fn=collact_batch)
        return train_dataloader, test_dataloader, MAX_LENGTH, embedding_size




In [11]:
# Loading sataset, use toy = True for obtaining a smaller dataset

train_dataset, test_dataset, num_words, input_size = get_data_set(batch_size)




In [23]:
# Special matrix multipication layer (like torch.Linear but can operate on arbitrary sized
# tensors and considers its last two indices as the matrix.)

class MatMul(nn.Module):
    def __init__(self, in_channels, out_channels, use_bias = True):
        super(MatMul, self).__init__()
        self.matrix = torch.nn.Parameter(torch.nn.init.xavier_normal_(torch.empty(in_channels,out_channels)),
                                         requires_grad=True)
        if use_bias:
            self.bias = torch.nn.Parameter(torch.zeros(1,1,out_channels), requires_grad=True)

        self.use_bias = use_bias

    def forward(self, x):
        x = x.to(DEVICE)
        x = torch.matmul(x,self.matrix)
        if self.use_bias:
            x = x+ self.bias
        return x


In [24]:
# Implements RNN Unit

class ExRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ExRNN, self).__init__()

        self.hidden_size = hidden_size
        self.sigmoid = torch.sigmoid
        self.tanh = torch.tanh

        # RNN Cell weights
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size).to(DEVICE)
        self.output_mlp = nn.Linear(hidden_size,output_size).to(DEVICE)

    def name(self):
        return "RNN"

    def forward(self, x, hidden_state): #/1,time,(100,100)

        x, hidden_state = x.to(DEVICE), hidden_state.to(DEVICE)
        # Implementation of RNN cell
        mid = torch.cat((x, hidden_state), 1)
        hidden = self.tanh(self.in2hidden(mid))
        output =self.output_mlp(hidden)
        return output, hidden

    def init_hidden(self, bs):
        return torch.zeros(bs, self.hidden_size).to(DEVICE)



In [14]:
# Implements GRU Unit

class ExGRU(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ExGRU, self).__init__()
        self.hidden_size = hidden_size
        self.sigmoid = torch.sigmoid
        self.tanh = torch.tanh
        self.update_gate = nn.Linear(input_size+hidden_size,hidden_size).to(DEVICE)
        self.reset_gate = nn.Linear(input_size+hidden_size,hidden_size).to(DEVICE)
        self.fc = nn.Linear(input_size + hidden_size, hidden_size).to(DEVICE)
        self.output_mlp = nn.Linear(hidden_size,output_size).to(DEVICE)

    def name(self):
        return "GRU"

    def forward(self, x, hidden_state):
        x, hidden_state = x.to(DEVICE), hidden_state.to(DEVICE)
        cat_tensor = torch.cat([hidden_state,x],dim=1)
        zt = self.sigmoid(self.update_gate(cat_tensor))
        rt = self.sigmoid(self.reset_gate(cat_tensor))
        mid = hidden_state*rt
        cat_mid = torch.cat([mid,x], dim=1)
        h_hat = self.tanh(self.fc(cat_mid))
        hidden = ((1-zt)*hidden_state) + (zt*h_hat)
        output = self.output_mlp(hidden)
        return output, hidden

    def init_hidden(self, bs):
        return torch.zeros(bs, self.hidden_size).to(DEVICE)




In [15]:
class ExMLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ExMLP, self).__init__()

        self.ReLU = torch.nn.ReLU()

        # Token-wise MLP network weights
        self.layer1 = MatMul(input_size,hidden_size).to(DEVICE)
        # additional layer(s)
        self.layer2 = nn.Linear(hidden_size,hidden_size).to(DEVICE)


    def name(self):
        return "MLP"

    def forward(self, x):
        x = x.to(DEVICE)
        # Token-wise MLP network implementation
        x = self.layer1(x)
        x = self.ReLU(x)
        # rest
        x = self.layer2(x)
        x = torch.sum(x, dim=1)

        return x




In [16]:
class ExLRestSelfAtten(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        pass # todo: fix
        super(ExLRestSelfAtten, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.sqrt_hidden_size = np.sqrt(float(hidden_size))
        self.ReLU = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(2)

        # Token-wise MLP + Restricted Attention network implementation

        self.layer1 = MatMul(input_size,hidden_size)
        self.W_q = MatMul(hidden_size, hidden_size, use_bias=False)
        # rest ...


    def name(self):
        return "MLP_atten"

    def forward(self, x):

        # Token-wise MLP + Restricted Attention network implementation

        x = self.layer1(x)
        x = self.ReLU(x)

        # generating x in offsets between -atten_size and atten_size
        # with zero padding at the ends

        padded = pad(x,(0,0,atten_size,atten_size,0,0))

        x_nei = []
        for k in range(-atten_size,atten_size+1):
            x_nei.append(torch.roll(padded, k, 1))

        x_nei = torch.stack(x_nei,2)
        x_nei = x_nei[:,atten_size:-atten_size,:]

        # x_nei has an additional axis that corresponds to the offset

        # Applying attention layer

        # query = ...
        # keys = ...
        # vals = ...


        return x, atten_weights


# prints portion of the review (20-30 first words), with the sub-scores each work obtained
# prints also the final scores, the softmaxed prediction values and the true label values

def print_review(rev_text, sbs1, sbs2, lbl1, lbl2):
    # Extract a portion of the review text (20-30 first words)
    review_portion = ' '.join(rev_text[:30])

    # Print review portion
    print(f"Review text (20-30 words): {review_portion}")

    # Print sub-scores for each word
    print(f"Sub-scores for class 1 (negative): {sbs1}")
    print(f"Sub-scores for class 2 (positive): {sbs2}")

    # Calculate final scores
    final_scores = torch.tensor([sbs1, sbs2])

    # Softmax to get probabilities
    probabilities = F.softmax(final_scores, dim=0)

    # Print final scores, softmaxed prediction values, and true label values
    print(f"Final scores: {final_scores}")
    print(f"Softmaxed predictions: {probabilities}")
    print(f"True label values: {lbl1}, {lbl2}")

In [17]:
# @title Run Model Function
# select model to use

def run_model(results, hidden_size=64, run_recurrent=True, use_RNN=True, reload_model=False, atten_size=0):
      if run_recurrent:
          if use_RNN:
              model = ExRNN(input_size, output_size, hidden_size)
          else:
              model = ExGRU(input_size, output_size, hidden_size)
      else:
          if atten_size > 0:
              model = ExLRestSelfAtten(input_size, output_size, hidden_size)
          else:
              model = ExMLP(input_size, output_size, hidden_size)

      print("Using model: " + model.name())

      if reload_model:
          print("Reloading model")
          model.load_state_dict(torch.load(model.name() + ".pth"))

      # move model to GPU if avilavble
      model.to(DEVICE)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

      train_loss = 1.0
      test_loss = 1.0

      train_losses = []
      test_losses = []
      accuracies = []

      # training steps in which a test step is executed every test_interval

      for epoch in range(num_epochs):

          itr = 0 # iteration counter within each epoch

          for labels, reviews, reviews_text in train_dataset:   # getting training batches

              # print(f"FOR DEBBUGING:: Labels shape: {labels.shape}")
              # print(f"FOR DEBBUGING:: Label: {labels}")

              itr = itr + 1

              if (itr + 1) % test_interval == 0:
                  test_iter = True
                  labels, reviews, reviews_text = next(iter(test_dataset)) # get a test batch
              else:
                  test_iter = False

              # Recurrent nets (RNN/GRU)

              if run_recurrent:
                  hidden_state = model.init_hidden(int(labels.shape[0]))

                  for i in range(num_words):
                      output, hidden_state = model(reviews[:,i,:], hidden_state)

              else:

              # Token-wise networks (MLP / MLP + Atten.)

                  sub_score = []
                  if atten_size > 0:
                      # MLP + atten
                      sub_score, atten_weights = model(reviews)
                  else:
                      # MLP
                      sub_score = model(reviews)

                  output = torch.mean(sub_score, 1)

                  # print(f"FOR DEBBUGING:: Output shape: {output.shape}, Labels shape: {labels.shape}")
                  if model.name() == "MLP":
                      labels = torch.argmax(labels, dim=1)
                      labels = labels.to(torch.float32)
                      labels.to(DEVICE)
                      # print(f"FOR DEBBUGING:: Label: {labels}")

              # cross-entropy loss

              loss = criterion(output, labels)

              # optimize in training iterations

              if not test_iter:
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

              # averaged losses
              if test_iter:
                  test_loss = 0.8 * float(loss.detach()) + 0.2 * test_loss
              else:
                  train_loss = 0.9 * float(loss.detach()) + 0.1 * train_loss

              if test_iter:
                train_losses.append(train_loss)
                test_losses.append(test_loss)


                # Calculate accuracy
                # Ensure the output and labels have compatible dimensions
                #print(f"Output shape before processing: {output.shape}")
                #print(f"Labels shape before processing: {labels.shape}")

                if len(output.shape) > 1 and output.shape[1] > 1:
                    _, predicted = torch.max(output, 1)
                else:
                    predicted = output

                # Convert labels to class indices if they are not already
                if len(labels.shape) > 1 and labels.shape[1] > 1:
                    labels = torch.argmax(labels, dim=1)

                # Print shapes after processing
                #print(f"Output shape after processing: {output.shape}")
                #print(f"Predicted shape: {predicted.shape}")
                #print(f"Labels shape after processing: {labels.shape}")

                # Print values to debug
                #print(f"Output: {output}")
                #print(f"Predicted: {predicted}")
                #print(f"Labels: {labels}")

                correct = (predicted == labels).sum().item()
                accuracy = correct / labels.size(0)
                accuracies.append(accuracy)

                # Print correct predictions and accuracy
                #print(f"Correct predictions: {correct}")
                #print(f"Accuracy: {accuracy}")


                print(
                      f"Epoch [{epoch + 1}/{num_epochs}], "
                      f"Step [{itr + 1}/{len(train_dataset)}], "
                      f"Train Loss: {train_loss:.4f}, "
                      f"Test Loss: {test_loss:.4f}"
                  )

                if not run_recurrent and reload_model: # don't prinr preview in train
                    nump_subs = sub_score.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    print_review(reviews_text[0], nump_subs[0], nump_subs[1], labels[0], labels[1])

                  # saving the model
                torch.save(model, model.name() + ".pth")
      results = {
        "train_losses": train_losses,
        "test_losses": test_losses,
        "accuracies": accuracies
      }

      return results

In [77]:
if __name__ == '__main__':
    results = {}
    hidden_sizes = [64, 128]
    for hidden_size in hidden_sizes:
        results[hidden_size] = run_model(results, hidden_size=hidden_size, run_recurrent=run_recurrent, use_RNN=use_RNN, reload_model=reload_model, atten_size=atten_size)
        plt.figure()
        plt.plot(results[hidden_size]['train_losses'], label='Train Loss')
        plt.plot(results[hidden_size]['test_losses'], label='Test Loss')
        plt.plot(results[hidden_size]['accuracies'], label='Accuracy')
        plt.title(f'Performance with hidden size of {hidden_size}')
        plt.xlabel('Iterations')
        plt.ylabel('Loss/Accuracy')
        plt.legend()
        plt.show()


Using model: MLP
Epoch [1/10], Step [50/938], Train Loss: 47.9626, Test Loss: 31.5902
Epoch [1/10], Step [100/938], Train Loss: 39.0839, Test Loss: 45.3888
Epoch [1/10], Step [150/938], Train Loss: 57.9602, Test Loss: 49.7720
Epoch [1/10], Step [200/938], Train Loss: 35.6862, Test Loss: 49.6095
Epoch [1/10], Step [250/938], Train Loss: 38.2480, Test Loss: 57.6064
Epoch [1/10], Step [300/938], Train Loss: 64.5071, Test Loss: 48.7661
Epoch [1/10], Step [350/938], Train Loss: 59.4570, Test Loss: 32.4287
Epoch [1/10], Step [400/938], Train Loss: 51.7095, Test Loss: 36.5872
Epoch [1/10], Step [450/938], Train Loss: 58.7373, Test Loss: 52.8906
Epoch [1/10], Step [500/938], Train Loss: 42.2301, Test Loss: 57.5773
Epoch [1/10], Step [550/938], Train Loss: 46.1389, Test Loss: 44.6670
Epoch [1/10], Step [600/938], Train Loss: 44.7617, Test Loss: 46.0143
Epoch [1/10], Step [650/938], Train Loss: 49.8106, Test Loss: 49.1451
Epoch [1/10], Step [700/938], Train Loss: 35.1072, Test Loss: 39.9918
Epoc

KeyError: 'train_losses'

<Figure size 640x480 with 0 Axes>

In [27]:
def inference_model(model_path, test_dataset, run_recurrent=True, num_words=None):
    # Load the model
    model = torch.load(model_path)
    model.eval()
    if model.name() == "MLP" or model.name() == "MLP_atten":
        pass
    results = []

    for review, label in zip(my_test_texts, my_test_labels):
        # Preprocess the review
        processed_review = preprocess_review(review)

        # Initialize hidden state
        if 'MLP' not in model_path:
          hidden = model.init_hidden(1)  # Batch size is 1 for inference

        # Run inference
          with torch.no_grad():
              output = None
              for i in range(processed_review.size(1)):  # Iterate over time steps
                  input_step = processed_review[:, i, :]
                  output, hidden = model(input_step, hidden)

        else:
            with torch.no_grad():
              # todo: fix this
              print("up to here")
              pass



        # Get the predicted label
        predicted_label = "pos" if output[0][0] > output[0][1] else "neg"

        results.append({
            "review": review,
            "true_label": label,
            "predicted_label": predicted_label,
            "confidence": max(output[0]).item()
        })

    return results

# Function to print results
def print_results(model_name, results):
    print(f"\nResults for {model_name}:")
    for result in results:
        print(f"Review: {result['review'][:150]}...")
        print(f"True label: {result['true_label']}")
        print(f"Predicted label: {result['predicted_label']}")
        print(f"Confidence: {result['confidence']:.4f}")
        print()

# Run inference for each model
model_paths = [
    # "RNN.pth",
    # "GRU.pth",
    "MLP.pth",
    # "MLP_atten.pth"
]

for model_path in model_paths:
    results = inference_model(model_path, None)  # We don't need test_dataset here
    print_results(model_path, results)

[[79.87416  77.12515  69.37811  65.93113  69.69392  80.95571  69.351074
  74.74272  68.603905 69.75887  73.839615 84.694084 81.30048  70.348114
  68.18561  70.47383  71.09858  74.94895  84.44992  78.81839  71.19244
  81.791306 65.64942  72.84935  83.37217  65.454796 65.78671  65.154724
  78.87431  74.85554  73.90263  65.66325  84.424805 80.14343  72.11169
  77.56879  67.77528  69.822876 68.93255  61.87874  84.751205 74.049164
  71.344795 70.73912  79.33252  82.92139  83.3423   83.68999  71.2025
  81.14024  80.11383  72.35251  83.63968  67.9016   81.466736 70.08443
  73.33458  64.50959  68.80419  84.809944 65.14062  81.6607   79.61394
  75.19978 ]]
tensor([[[ 0.2831,  0.3328,  0.5000,  ...,  0.0315, -0.2246,  0.4851],
         [ 0.3045, -0.1963,  0.2023,  ..., -0.1839, -0.1243,  0.2747],
         [ 0.1571,  0.6561,  0.0021,  ..., -0.6061,  0.7100,  0.4147],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.

UnboundLocalError: local variable 'output' referenced before assignment

review type: <class 'ellipsis'>


TypeError: expected string or bytes-like object